In [120]:
import random
import numpy as np

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from sklearn.metrics import classification_report
from itertools import product
from sklearn.metrics import accuracy_score as accuracy
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.model_selection import train_test_split
import math

np.random.seed(1337)  # for reproducibility
input_dim = 784
num_classses = 10
img_rows, img_cols = 28, 28
num_classes = 10

In [2]:
def preprocess_data(X, y):
    #create tensor variant of 2D images
    if K.image_data_format() == 'channels_first':
        X = X.reshape(X.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X = X.reshape(X.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    X = X.astype('float32') / 255
    #X = (X - np.mean(X)) / np.std(X)
    # convert class vectors to binary class matrices
    #y = keras.utils.to_categorical(y, num_classes)
    return X, y, input_shape

data=np.load("FashionData/FashionPDEngDM.npz")

## Labeled training set for classes 1,2,3,8,9 (30000 samples)
x_train_12389_labeled = data["x_train_12389_labeled"]
y_train_12389_labeled = data["y_train_12389_labeled"]

## Labeled training set for classes 0,4,5,6,7 (just 5 samples)
x_train_04567_labeled=data["x_train_04567_labeled"]
y_train_04567_labeled=data["y_train_04567_labeled"]

## Unlabeled training set for classes 0,4,5,6,7 (29992 samples)>
x_train_04567_unlabeled=data["x_train_04567_unlabeled"]

## Labeled test set for classes 1,2,3,8,9
x_test_12389=data["x_test_12389"]
y_test_12389=data["y_test_12389"]

##Labeled test set for classes 0,4,5,6,7 (this is where we are interested to obtain the highest accuracy possible - project goal)
x_test_04567=data["x_test_04567"]
y_test_04567=data["y_test_04567"]

x_train_12389_labeled, y_train_12389_labeled, input_shape = preprocess_data(x_train_12389_labeled, y_train_12389_labeled)
x_test_12389, y_test_12389, _ = preprocess_data(x_test_12389, y_test_12389)

x_train_04567_labeled, y_train_04567_labeled, input_shape = preprocess_data(x_train_04567_labeled, y_train_04567_labeled)
x_test_04567, y_test_04567, _ = preprocess_data(x_test_04567, y_test_04567)

In [68]:
# image augumentation
rotation_range = 50
width_shift_range = 0.5
height_shift_range = 0.5
shear_range = 0.5
horizontal_flip = True
vertical_flip = True
fill_mode = 'nearest'
zoom_range = 0.2
x_train_04567_aug = ImageDataGenerator(
    rotation_range = rotation_range,
    shear_range = shear_range,
    width_shift_range= width_shift_range,
    height_shift_range= height_shift_range,
    horizontal_flip = horizontal_flip,
    vertical_flip = vertical_flip,
    zoom_range = zoom_range,
    fill_mode = fill_mode
)
x_train_04567_aug.fit(x_train_04567_labeled)

In [121]:
# display and generate augmented data 
def get_augmented_data(generator, x, y, N, visualize = False):
    """Generate augmented data.
    
    Params: 
    - generator: ImageGenerator 
    - x: X data
    - y: y data
    - N: number of images to generates per class
    - visualize: whether to visualize the augmented images
    
    Return: (x, y) augmneted data
    """
    
    x_train_04567_aug_generated = []
    y_train_04567_aug_generated = []

    data_len = len(x)    
    data_gen = generator.flow(x, y, batch_size = data_len)
    num_batch_print = 2
    num_columns = 2

    for i, data_batch in enumerate(data_gen):    
        if N <= i: # in every iteration, one instance per class is generated
            break
       
        x_batch, y_batch = data_batch
        #save data
        x_train_04567_aug_generated.append(x_batch)
        y_train_04567_aug_generated.append(y_batch)
        
        if visualize: # visualize the augmented results
            f, axarr = plt.subplots(data_len, num_columns, figsize = (5, 5))
            for j in range(len(x_batch)):
                ground_ind = np.where(y == y_batch[j])
                axarr[j, 0].imshow(x[ground_ind].reshape((28,28)), cmap='gray')   
                axarr[j, 1].imshow(x_batch[j].reshape((28,28)), cmap='gray')
                axarr[j, 0].axis('off')
                axarr[j, 1].axis('off')
            print(i)
            plt.show()
    
    x_train_04567_aug_generated[-1]
    # reshaping such that the array has the shape of (# data, 28, 28, 1)
    x_train_04567_aug_generated = np.array(x_train_04567_aug_generated).reshape((-1, 28, 28, 1)) 
  
    #reshaping to shape of (# data)
    y_train_04567_aug_generated = np.array(y_train_04567_aug_generated).reshape((-1)) 
#    modulo = N % data_len
#     if modulo != 0:
#         x_train_04567_aug_generated = x_train_04567_aug_generated[0 : -(data_len-modulo)]
#         y_train_04567_aug_generated = y_train_04567_aug_generated[0 : -(data_len-modulo)]
#         print("Readjusting the shape. The shape is %s " % (x_train_04567_aug_generated.shape, ))
        #print("Warning: The data lenght will be slightly bigger than N, because N is not divadable by size of data.")
    return x_train_04567_aug_generated, y_train_04567_aug_generated

x_train_04567_aug_generated, y_train_04567_aug_generated = get_augmented_data(generator = x_train_04567_aug, 
                                                                              x = x_train_04567_labeled,
                                                                              y = y_train_04567_labeled,
                                                                              N = 3000)

In [122]:
# create training+test positive and negative pairs
def create_pairs(x, class_indices, num_classes):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs. 

    Maximally creates (min(class_size) - 1) * num_classes pairs.
    '''
    pairs = []
    labels = []    
    n = min([len(class_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = class_indices[d][i], class_indices[d][i+1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes # this guarantees that the same class will not be selected
            z1, z2 = class_indices[d][i], class_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

ind_04567 = [0,4,5,6,7]
class_indices = [np.where(y_train_04567_aug_generated == i)[0] for i in ind_04567]
x_pairs, y_pairs = create_pairs(x_train_04567_aug_generated, class_indices, int(num_classes / 2))
x_train_pairs, x_val_pairs, y_train_pairs, y_val_pairs = train_test_split(x_pairs, y_pairs, test_size=0.3, random_state=42)
print("Number of training pairs is %d " % x_train_pairs.shape[0])
print("Number of val pairs is %d " % x_val_pairs.shape[0])

Number of training pairs is 20993 
Number of val pairs is 8997 


In [140]:
# model definition
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    #model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.5))
    return model

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)  

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1  

base_network = create_base_network(input_shape)

input_a = Input(shape = input_shape)
input_b = Input(shape = input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape = abs_diff_output_shape)([processed_a, processed_b])

flattened_weighted_distance = Dense(1, activation = 'sigmoid')(abs_diff)

siamese_model = Model(input=[input_a, input_b], output = flattened_weighted_distance)

C:\Users\azika\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [141]:
# train
def step_decay(epoch):
    '''Learning rate step decay following the original paper.'''
    initial_lrate = 0.001
    drop = 0.99
    epochs_drop = 1
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate


nb_epoch = 50
optimizer = Adam()
siamese_model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

siamese_model.fit([x_train_pairs[:, 0], x_train_pairs[:, 1]], #pairs 
          y_train_pairs, #labels of the pairs
          callbacks = callbacks_list,
          batch_size=128,
          nb_epoch=nb_epoch)
siamese_model.save('siamese_model_without_dense.h5')

C:\Users\azika\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/50
20993/20993 [==============================] - 80s - loss: 0.6934 - acc: 0.5159    
Epoch 2/50
20993/20993 [==============================] - 73s - loss: 0.6888 - acc: 0.5380    
Epoch 3/50
20993/20993 [==============================] - 72s - loss: 0.6692 - acc: 0.5875    
Epoch 4/50
20993/20993 [==============================] - 72s - loss: 0.6536 - acc: 0.6127    
Epoch 5/50
20993/20993 [==============================] - 72s - loss: 0.6400 - acc: 0.6270    
Epoch 6/50
20993/20993 [==============================] - 72s - loss: 0.6342 - acc: 0.6337    
Epoch 7/50
20993/20993 [==============================] - 72s - loss: 0.6318 - acc: 0.6360    
Epoch 8/50
20993/20993 [==============================] - 72s - loss: 0.6170 - acc: 0.6496    
Epoch 9/50
20993/20993 [==============================] - 75s - loss: 0.6281 - acc: 0.6423    
Epoch 10/50
20993/20993 [==============================] - 74s - loss: 0.6029 - acc: 0.6654    
Epoch 11/50
20993/20993 [========================

In [142]:
# validation accuracy
y_val_pred_proba = siamese_model.predict([x_val_pairs[:, 0], x_val_pairs[:, 1]])

In [143]:
# validation accuracy
y_val_pred_class = y_val_pred_proba > 0.5 
y_val_pred_class = y_val_pred_class.reshape(-1)
val_accur = np.sum(y_val_pred_class == y_val_pairs) / len(y_val_pairs)
print('* Accuracy of classifying the val set: {:.2%}'.format(val_accur))

* Accuracy of classifying the val set: 69.46%


In [144]:
# generate examples (k per class) that will be couples with test data
k = 5 # k from from analogy of k-means
x_test_04567_aug_generated, y_test_04567_aug_generated = get_augmented_data(generator = x_train_04567_aug, 
                                                                            x = x_train_04567_labeled,
                                                                            y = y_train_04567_labeled,
                                                                            N = k)

In [145]:
# create pairs
x_test_pairs = np.array(list(product(x_test_04567, x_test_04567_aug_generated)))

# predict the probability of a pair being similar. 
y_test_pred_prob = siamese_model.predict([x_test_pairs[:, 0], x_test_pairs[:, 1]])

# reshaping such that each row contains k*number of class probabilities 
y_test_pred_prob = y_test_pred_prob.reshape(len(x_test_04567), len(x_test_04567_aug_generated))

# getting top k prediction for a class
y_test_pred_class_inds = y_test_pred_prob.argsort(axis = 1)[:, -k:]

# like argsort but only for the highest score
#y_test_pred_class_inds = np.argmax(y_test_pred_prob, axis = 1)

# reverse order -> the highest prob is on the first place (This does not work! it reserve the data )
#y_test_pred_class_inds = y_test_pred_class_inds[::-1] 

In [146]:
# generating class prediction
y_test_pred_k = np.array([mode(y_test_04567_aug_generated[y_test_pred_class_inds[i]]) 
                                for i in range(len(y_test_pred_class_inds))])[:, 0]
y_test_pred_k = y_test_pred_k.reshape(-1)

In [147]:
# how many prediction per class
np.unique(y_test_pred_k, return_counts= True)

(array([0, 4, 5, 6, 7]), array([1051, 1256,  508, 1837,  348], dtype=int64))

In [148]:
# final accuracy
test_accur = np.sum(y_test_pred_k == y_test_04567) / len(y_test_04567)
print('* Accuracy of classifying the test set: {:.2%}'.format(test_accur))

* Accuracy of classifying the test set: 29.12%
